# hiseq.align


The standard port for aligner

## 1 Requirements  


### 1.1 Arguments

  - input: str 
  - index: str
  - outdir: pwd/str
  - unique: True/False
  - aligner: (guess aligner) 
  - smp_name: None/str
  - index_name: None/str
  - threads: 4/int
  - overwrite: True/False
  - extra_para: (in dict, toml)


### 1.2 Output

  - config.toml
  - bam  
  - log  
  - stat 
  - cmd.sh 
  - unmap  
  - flagstat 


### 1.3 Directory structure 

  - outdir/smp_name/index_name/files



## 2. Interpreter 



### 2.1 Functions 

  - unique|multiple, determine the aligner 
  

### 2.2 Output

  - required arguments for `Align` 




## 3 Aligner


### 3.1 Bowtie   

```
# common: -p 4 --mm
# default: k=1
bowtie -k 1 -S -v 2 --best --un unmap.fq -x index se.fq 1> align.sam 2> align.log
bowtie -k 1 -S -v 2 --best --un unmap.fq -x index -1 pe_1.fq -2 pe_2.fq 1> align.sam 2> align.log

# unique
bowtie -m 1 -S -v 2 --best --un unmap.fq -x index se.fq 1> align.sam 2> align.log
bowtie -m 1 -S -v 2 --best --un unmap.fq -x index -1 pe_1.fq -2 pe_2.fq 1> align.sam 2> align.log
```

### 3.2 Bowtie2 

```
# common: -p 4 --mm
# default:
bowtie2 --very-sensitive --no-unal -x index -U se.fq --un unmap.fq 1> align.sam 2> align.log
bowtie2 --very-sensitive --no-unal --no-mixed --no-discordant --un-conc unmap.fq 1> align.sam 2> align.log

# unique
# https://www.biostars.org/p/101533/#101537, -q 5,10
samtools view -bhs -q 10 align.bam 


# extra parameters
-I minimum fragment length (0) 
-X maximum fragment length (500) 
```


### 3.3 STAR 

```
# common
# default:
STAR \
    --genomeLoad NoSharedMemory \
    --runMode alignReads \
    --genomeDir index \
    --readFilesIn pe_1 pe_2 \
    --readFilesCommand zcat \
    --outFileNamePrefix smp_name \
    --runThreadN 4 \
    --limitBAMsortRAM 10000000000 \
    --outSAMtype BAM SortedByCoordinate \
    --outFilterMismatchNoverLmx 0.07 \
    --seedSearchStartLmax 20 \
    --outReadsUnmapped Fastx 

# unique:
--outFilterMultimapNmax 1

# small genome (default: 50)
--seedPerWindowNmax 5


# For sharing memory in STAR
# by Devon Ryan: https://www.biostars.org/p/260069/#260077 
# by Dobin: https://github.com/alexdobin/STAR/pull/26
# --genomeLoad
```


















Updates:

1. style the code with PEP8: https://www.python.org/dev/peps/pep-0008/

in brief:

1 indentation: 4 spaces  
    1.1 Aligned with opening delimiter
2 Maximum Line Length: 79 characters 
    2.1 Multi lines, operator at end
3 Blank Lines
    3.1 Two blank lines surround top-level function/class  
    3.2 Single blank line, in class  
    3.3 
4 White spaces  
    4.1 no spaces after/before brackets/braces  
    4.2 single space on both sides of binary operator   
    4.3 no spaces around '=' for keyword argument  

5 Naming styles

5.1 lower_case_with_underscores, 

6 Package and Module Names 

6.1 Modules should have short, all-lowercase names

7 Class names should normally use the CapWords convention. 

8 Function and Variable Names,  lowercase, with words separated by underscores as necessary to improve readability.

9 Use `is`, `is not`

10 Use ''.startswith(), ''.endswith()



















In [ ]:
%reload_ext autoreload
%autoreload 2

import aligner_index
from aligner_index import *
from utils import *

args = {
    'genome': 'mm10',
    'aligner': 'bowtie',
    # 'spikein': 'dm3',
    # 'to_rRNA': True,
    'to_chrM': True,
    # 'to_MT_trRNA': True,
    'verbose': True,
}

# check_index_args(**args)

# fetch_index('dm6', 'chrM', 'bowtie')

# x = '/home/wangming/data/genome/dm6/bowtie_index/chrM'
# aligner = 'bowtie'

# p = AlignIndex(x, aligner)

# p.is_bowtie_index()

In [26]:
%reload_ext autoreload
%autoreload 2

from bowtie2 import *

args = {
    'fq1': '/data/yulab/wangming/work/devel_pipeline/hiseq/rnaseq/data/pe_control_rep1_1.fq.gz',
    'fq2': '/data/yulab/wangming/work/devel_pipeline/hiseq/rnaseq/data/pe_control_rep1_2.fq.gz',
    'index': '/home/wangming/data/genome/dm6/bowtie2_index/chrM',
    'outdir': 'test',
    'genome': 'dm6',
    'threads': 24,
    'unique_only': True,
}

# a = BowtieConfig(**args)
# check_fx_args(args['fq1'], args['fq2'])


a = Bowtie2(**args)
a.run()



# x = 'test/pe_control_rep1/chrM/pe_control_rep1.align.log'
# parse_bowtie2(x)

[2021-03-17 12:37:46 INFO] Bowtie2() skipped, file exists: /data/yulab/wangming/work/wmlib/hiseq/hiseq/aligner/test/pe_control_rep1/chrM/pe_control_rep1.bam


('/data/yulab/wangming/work/wmlib/hiseq/hiseq/aligner/test/pe_control_rep1/chrM/pe_control_rep1.bam',
 '/data/yulab/wangming/work/wmlib/hiseq/hiseq/aligner/test/pe_control_rep1/chrM/pe_control_rep1.unmap.1.fastq',
 '/data/yulab/wangming/work/wmlib/hiseq/hiseq/aligner/test/pe_control_rep1/chrM/pe_control_rep1.unmap.2.fastq')

In [ ]:
%reload_ext autoreload
%autoreload 2
from bowtie2 import *

help(check_file)